run_models: 更新config.py中model对应的name，以实现对应model的运行

In [5]:
#NeuMF
!python main.py --batch_size=256 --lr=0.001 --factor_num=8

The time elapse of epoch 000 is: 00: 01: 52
HR: 0.592	NDCG: 0.334
The time elapse of epoch 001 is: 00: 01: 49
HR: 0.636	NDCG: 0.366
The time elapse of epoch 002 is: 00: 01: 51
HR: 0.652	NDCG: 0.376
The time elapse of epoch 003 is: 00: 01: 49
HR: 0.664	NDCG: 0.388
The time elapse of epoch 004 is: 00: 01: 52
HR: 0.663	NDCG: 0.391
The time elapse of epoch 005 is: 00: 01: 53
HR: 0.671	NDCG: 0.398
The time elapse of epoch 006 is: 00: 01: 48
HR: 0.670	NDCG: 0.397
The time elapse of epoch 007 is: 00: 01: 56
HR: 0.672	NDCG: 0.399
The time elapse of epoch 008 is: 00: 01: 48
HR: 0.677	NDCG: 0.402
The time elapse of epoch 009 is: 00: 01: 49
HR: 0.679	NDCG: 0.403
The time elapse of epoch 010 is: 00: 01: 58
HR: 0.676	NDCG: 0.403
The time elapse of epoch 011 is: 00: 01: 37
HR: 0.677	NDCG: 0.404
The time elapse of epoch 012 is: 00: 01: 44
HR: 0.676	NDCG: 0.405
The time elapse of epoch 013 is: 00: 01: 45
HR: 0.678	NDCG: 0.406
The time elapse of epoch 014 is: 00: 01: 37
HR: 0.679	NDCG: 0.403
The time e

In [4]:
#GMF
!python main.py --batch_size=256 --lr=0.001 --factor_num=8

The time elapse of epoch 000 is: 00: 01: 24
HR: 0.497	NDCG: 0.276
The time elapse of epoch 001 is: 00: 01: 22
HR: 0.560	NDCG: 0.314
The time elapse of epoch 002 is: 00: 01: 24
HR: 0.594	NDCG: 0.335
The time elapse of epoch 003 is: 00: 01: 21
HR: 0.613	NDCG: 0.346
The time elapse of epoch 004 is: 00: 01: 24
HR: 0.622	NDCG: 0.354
The time elapse of epoch 005 is: 00: 01: 22
HR: 0.628	NDCG: 0.359
The time elapse of epoch 006 is: 00: 01: 27
HR: 0.629	NDCG: 0.363
The time elapse of epoch 007 is: 00: 01: 21
HR: 0.631	NDCG: 0.363
The time elapse of epoch 008 is: 00: 01: 27
HR: 0.634	NDCG: 0.365
The time elapse of epoch 009 is: 00: 01: 20
HR: 0.634	NDCG: 0.366
The time elapse of epoch 010 is: 00: 01: 26
HR: 0.637	NDCG: 0.366
The time elapse of epoch 011 is: 00: 01: 20
HR: 0.637	NDCG: 0.365
The time elapse of epoch 012 is: 00: 01: 26
HR: 0.633	NDCG: 0.367
The time elapse of epoch 013 is: 00: 01: 19
HR: 0.636	NDCG: 0.368
The time elapse of epoch 014 is: 00: 01: 26
HR: 0.636	NDCG: 0.368
The time e

In [13]:
#MLP 1
!python main.py --batch_size=256 --lr=0.001 --factor_num=8 --num_layers=1

The time elapse of epoch 000 is: 00: 01: 20
HR: 0.466	NDCG: 0.258
The time elapse of epoch 001 is: 00: 01: 35
HR: 0.481	NDCG: 0.263
The time elapse of epoch 002 is: 00: 01: 23
HR: 0.500	NDCG: 0.277
The time elapse of epoch 003 is: 00: 01: 28
HR: 0.514	NDCG: 0.284
The time elapse of epoch 004 is: 00: 01: 27
HR: 0.532	NDCG: 0.290
The time elapse of epoch 005 is: 00: 01: 24
HR: 0.545	NDCG: 0.303
The time elapse of epoch 006 is: 00: 01: 33
HR: 0.553	NDCG: 0.309
The time elapse of epoch 007 is: 00: 01: 21
HR: 0.561	NDCG: 0.312
The time elapse of epoch 008 is: 00: 01: 31
HR: 0.571	NDCG: 0.316
The time elapse of epoch 009 is: 00: 01: 25
HR: 0.571	NDCG: 0.319
The time elapse of epoch 010 is: 00: 01: 27
HR: 0.576	NDCG: 0.323
The time elapse of epoch 011 is: 00: 01: 32
HR: 0.576	NDCG: 0.326
The time elapse of epoch 012 is: 00: 01: 19
HR: 0.581	NDCG: 0.326
The time elapse of epoch 013 is: 00: 01: 33
HR: 0.584	NDCG: 0.329
The time elapse of epoch 014 is: 00: 01: 21
HR: 0.583	NDCG: 0.330
The time e

In [10]:
#MLP 2
!python main.py --batch_size=256 --lr=0.001 --factor_num=8 --num_layers=2

The time elapse of epoch 000 is: 00: 01: 29
HR: 0.495	NDCG: 0.274
The time elapse of epoch 001 is: 00: 01: 27
HR: 0.558	NDCG: 0.309
The time elapse of epoch 002 is: 00: 01: 33
HR: 0.572	NDCG: 0.322
The time elapse of epoch 003 is: 00: 01: 26
HR: 0.594	NDCG: 0.332
The time elapse of epoch 004 is: 00: 01: 29
HR: 0.599	NDCG: 0.335
The time elapse of epoch 005 is: 00: 01: 32
HR: 0.608	NDCG: 0.346
The time elapse of epoch 006 is: 00: 01: 26
HR: 0.615	NDCG: 0.350
The time elapse of epoch 007 is: 00: 01: 33
HR: 0.622	NDCG: 0.356
The time elapse of epoch 008 is: 00: 01: 27
HR: 0.624	NDCG: 0.357
The time elapse of epoch 009 is: 00: 01: 29
HR: 0.628	NDCG: 0.362
The time elapse of epoch 010 is: 00: 01: 31
HR: 0.634	NDCG: 0.365
The time elapse of epoch 011 is: 00: 01: 26
HR: 0.628	NDCG: 0.361
The time elapse of epoch 012 is: 00: 01: 32
HR: 0.631	NDCG: 0.366
The time elapse of epoch 013 is: 00: 01: 31
HR: 0.634	NDCG: 0.366
The time elapse of epoch 014 is: 00: 01: 27
HR: 0.633	NDCG: 0.370
The time e

In [2]:
#MLP 3
!python main.py --batch_size=256 --lr=0.001 --factor_num=8 --num_layers=3

The time elapse of epoch 000 is: 00: 01: 33
HR: 0.523	NDCG: 0.293
The time elapse of epoch 001 is: 00: 01: 33
HR: 0.585	NDCG: 0.331
The time elapse of epoch 002 is: 00: 01: 40
HR: 0.610	NDCG: 0.346
The time elapse of epoch 003 is: 00: 01: 30
HR: 0.630	NDCG: 0.361
The time elapse of epoch 004 is: 00: 01: 39
HR: 0.642	NDCG: 0.369
The time elapse of epoch 005 is: 00: 01: 39
HR: 0.646	NDCG: 0.376
The time elapse of epoch 006 is: 00: 01: 32
HR: 0.646	NDCG: 0.377
The time elapse of epoch 007 is: 00: 01: 38
HR: 0.656	NDCG: 0.385
The time elapse of epoch 008 is: 00: 01: 33
HR: 0.658	NDCG: 0.384
The time elapse of epoch 009 is: 00: 01: 35
HR: 0.657	NDCG: 0.383
The time elapse of epoch 010 is: 00: 01: 39
HR: 0.661	NDCG: 0.386
The time elapse of epoch 011 is: 00: 01: 40
HR: 0.662	NDCG: 0.388
The time elapse of epoch 012 is: 00: 01: 37
HR: 0.664	NDCG: 0.392
The time elapse of epoch 013 is: 00: 01: 36
HR: 0.666	NDCG: 0.391
The time elapse of epoch 014 is: 00: 01: 33
HR: 0.665	NDCG: 0.391
The time e

In [3]:
#MLP 4
!python main.py --batch_size=256 --lr=0.001 --factor_num=8 --num_layers=4

The time elapse of epoch 000 is: 00: 01: 42
HR: 0.541	NDCG: 0.303
The time elapse of epoch 001 is: 00: 01: 44
HR: 0.603	NDCG: 0.343
The time elapse of epoch 002 is: 00: 01: 38
HR: 0.632	NDCG: 0.362
The time elapse of epoch 003 is: 00: 01: 42
HR: 0.643	NDCG: 0.372
The time elapse of epoch 004 is: 00: 01: 43
HR: 0.659	NDCG: 0.384
The time elapse of epoch 005 is: 00: 01: 37
HR: 0.663	NDCG: 0.391
The time elapse of epoch 006 is: 00: 01: 43
HR: 0.675	NDCG: 0.397
The time elapse of epoch 007 is: 00: 01: 42
HR: 0.677	NDCG: 0.398
The time elapse of epoch 008 is: 00: 01: 33
HR: 0.681	NDCG: 0.404
The time elapse of epoch 009 is: 00: 01: 46
HR: 0.687	NDCG: 0.407
The time elapse of epoch 010 is: 00: 01: 40
HR: 0.684	NDCG: 0.405
The time elapse of epoch 011 is: 00: 01: 37
HR: 0.684	NDCG: 0.406
The time elapse of epoch 012 is: 00: 01: 45
HR: 0.684	NDCG: 0.407
The time elapse of epoch 013 is: 00: 01: 41
HR: 0.684	NDCG: 0.406
The time elapse of epoch 014 is: 00: 01: 37
HR: 0.681	NDCG: 0.404
The time e

In [9]:
#MLP-0
!python main.py --batch_size=256 --lr=0.001 --factor_num=8

The time elapse of epoch 000 is: 00: 01: 27
HR: 0.448	NDCG: 0.250
The time elapse of epoch 001 is: 00: 01: 30
HR: 0.455	NDCG: 0.253
The time elapse of epoch 002 is: 00: 01: 20
HR: 0.452	NDCG: 0.251
The time elapse of epoch 003 is: 00: 01: 35
HR: 0.451	NDCG: 0.250
The time elapse of epoch 004 is: 00: 01: 18
HR: 0.451	NDCG: 0.251
The time elapse of epoch 005 is: 00: 01: 23
HR: 0.451	NDCG: 0.252
The time elapse of epoch 006 is: 00: 01: 27
HR: 0.450	NDCG: 0.251
The time elapse of epoch 007 is: 00: 01: 27
HR: 0.449	NDCG: 0.250
The time elapse of epoch 008 is: 00: 01: 29
HR: 0.450	NDCG: 0.251
The time elapse of epoch 009 is: 00: 01: 25
HR: 0.452	NDCG: 0.252
The time elapse of epoch 010 is: 00: 01: 11
HR: 0.450	NDCG: 0.252
The time elapse of epoch 011 is: 00: 01: 13
HR: 0.451	NDCG: 0.252
The time elapse of epoch 012 is: 00: 01: 23
HR: 0.451	NDCG: 0.251
The time elapse of epoch 013 is: 00: 01: 26
HR: 0.453	NDCG: 0.252
The time elapse of epoch 014 is: 00: 01: 30
HR: 0.450	NDCG: 0.251
The time e